### Imports

In [ ]:
import ck.kernel as ck
import json
import os

In [ ]:
print "Collective Knowledge: v%s" % ck.version({})['version_str']

### Find results

In [ ]:
r=ck.access({
  'action'      : 'list_points',
  'module_uoa'  : 'experiment',
  'data_uoa'    : 'explore-clsmith-datasets-any'
})

if r['return']>0:
  print ("Error: %s" % r['error'])
  exit(1)

path   = r['path']
points = r['points']

### Analyse results

In [ ]:
failures = {}
failures['num_total'] = 0
failures['num_timeouts'] = 0
failures['num_nonuniques'] = 0
failures['num_enqueue_errors'] = 0
failures['num_assertions'] = 0
failures['assertions'] = { }

In [ ]:
def analyse(failures, point):
    test   = point.get('##choices#dataset_file#min')
    result = point.get('##characteristics#run#result_string#min')
    reason = point.get('##characteristics#run#fail_reason#min')
    stderr = point.get('##characteristics#run#stderr#min')
    stdout = point.get('##characteristics#run#stdout#min')

    num_unique    = len(set(result.split(','))) if result != None else 0
    timeout_error = reason and (reason == 'process timed out and had been terminated')
    enqueue_error = ('Error enqueueing kernel: -5' in stdout)
    assertion     = ('Assertion' in stderr)

    if num_unique > 1:
        failures['num_nonuniques'] += 1

    if timeout_error:
        failures['num_timeouts'] += 1
        
    if enqueue_error:
        failures['num_enqueue_errors'] += 1
        
    if assertion:
        failures['num_assertions'] += 1
        failures['assertions'][test] = stderr

    print '== %s =='       % test
    print 'unique results: %s' % num_unique
    print 'failure reason: %s' % reason
    print 'stderr: %s' % stderr
    print 'stdout: %s' % stdout

    if ((num_unique != 1) or (reason != None) or (stderr != '')):
        failures['num_total'] += 1
        print '== FAILED =='
    else:
        print '== PASSED =='
        
    print

In [ ]:
# Analyse each experiment point.
for point in points:
    with open(os.path.join(path, 'ckp-%s.flat.json' % point)) as point_file:
        point_data = json.load(point_file)
    analyse(failures, point_data)

### Print summary

In [ ]:
print 'TOTAL NUMBER OF FAILURES: %d' % failures['num_total']
print '- NUMBER OF TIMEOUTS: %d' % failures['num_timeouts']
print '- NUMBER OF NON-UNIQUE RESULTS: %d' % failures['num_nonuniques']
print '- NUMBER OF ENQUEUE ERRORS: %d'% failures['num_enqueue_errors']
print '- NUMBER OF COMPILER ASSERTIONS: %d' % failures['num_assertions']
if failures['num_assertions'] > 0:
    print
    print 'ALL COMPILER ASSERTIONS:'
    for test, assertion in failures['assertions'].iteritems():
        print '== %s ==' % test
        print assertion